In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing


%matplotlib inline

In [4]:
types = {
    'ИНН': np.int64,
}
dfs = pd.read_csv(r'data\data.csv', dtype=types)
dfs.head()

,KBK,ОКВЭД2,"Вид деятельности, основной ТАСС","Вид деятельности, дополнительный ТАСС",Организационно правовая форма,Атрибуты предприятия,Отрасль,ОГРН,ИНН,КПП,Регион
0,02004111950168580812,NaN,Торговля оптовая производственным электротехни...,"Производство прочих химических продуктов, не в...",НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],NaN,Радиоэлектронная промышленность [258],1127847272629,7814536770,NaN,Санкт-Петербург [3522]
1,02004121840168844452,Производство оружия и боеприпасов [27574],"Строительство кораблей, судов и плавучих конст...",Производство оружия и боеприпасов [27575] / Пр...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация ОПК [8780665] / Системообразующее ...,Судостроительная промышленность [262],1079847085966,7838395215,783801001.0,Санкт-Петербург [3522]
2,73004122140062870451,Научные исследования и разработки в области ес...,Научные исследования и разработки в области ес...,Производство автоматических космических аппара...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Системообразующее предприятие [8690579] / Орга...,Транспортное машиностроение [264],1082452000290,2452034898,245201001.0,Красноярский край [3496]
3,02006051610166750811,Производство автотранспортных средств [27850] ...,Производство автотранспортных средств [27850],"Производство двигателей и турбин, кроме авиаци...",ОБЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ [7026...,NaN,Автомобильная промышленность [246],1023900987626,3906092856,NaN,Калининградская область [3485]
4,02004121616713810242,Производство автотранспортных средств [27850],Производство автотранспортных средств [27850],Торговля оптовая легковыми автомобилями и легк...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Системообразующее предприятие [8690579],Автомобильная промышленность [246],1027739178202,7709259743,772301001.0,Москва [3505]


In [15]:
with open(r'kbk.json', 'r') as f:
    index_to_kbk = json.load(f)
    kbk_to_index = {val: int(key) for key, val in index_to_kbk.items()}

In [74]:
with open(r'acting.json', 'r') as f:
    index_to_acting = json.load(f)
    acting_to_index = {val: int(key) for key, val in index_to_acting.items()}

In [75]:
with open(r'attr.json', 'r') as f:
    index_to_attr = json.load(f)
    attr_to_index = {val: int(key) for key, val in index_to_attr.items()}

In [92]:
with open(r'forma.json', 'r') as f:
    index_to_forma = json.load(f)
    forma_to_index = {val: int(key) for key, val in index_to_forma.items()}

In [71]:
with open(r'ocved.json', 'r') as f:
    index_to_ocved = json.load(f)
    ocved_to_index = {val: int(key) for key, val in index_to_ocved.items()}

In [76]:
with open(r'otrasl.json', 'r') as f:
    index_to_otrasl = json.load(f)
    otrasl_to_index = {val: int(key) for key, val in index_to_otrasl.items()}

In [93]:
with open(r'region.json', 'r') as f:
    index_to_region = json.load(f)
    region_to_index = {val: int(key) for key, val in index_to_region.items()}

In [108]:
def get_data(df, split=None):
    x = []
    y = []
    for _, line in df.iterrows():
        tmp = []
        
        # ОКВЭД2
        if type(line['ОКВЭД2']) != float:
            ocved = line['ОКВЭД2'].split(' / ')[:5]
            tmp += [ocved_to_index[i] for i in ocved]
            if len(ocved) < 5:
                tmp += [0] * (5-len(ocved))
        else:
            tmp += [ocved_to_index['-1']] + [0] * 4
        
        
        # основной ТАСС
        if type(line['Вид деятельности, основной ТАСС']) != float:
            tmp += [acting_to_index[line['Вид деятельности, основной ТАСС']]]
        else:
            tmp += [acting_to_index['-1']]
        
        # дополнительный ТАСС
        if type(line['Вид деятельности, дополнительный ТАСС']) != float:
            dop = line['Вид деятельности, дополнительный ТАСС'].split(' / ')[:80]
            tmp += [acting_to_index[i] for i in dop]
            if len(dop) < 80:
                tmp += [0] * (80-len(dop))
        else:
            tmp += [acting_to_index['-1']] + [0] * 79
            
        # Атрибуты предприятия
        if type(line['Атрибуты предприятия']) != float:
            attrs = line['Атрибуты предприятия'].split(' / ')[:3]
            tmp += [attr_to_index[i] for i in attrs]
            if len(attrs) < 3:
                tmp += [0] * (3-len(attrs))
        else:
            tmp += [attr_to_index['-1']] + [0] * 2
            
        # Отрасль
        if type(line['Отрасль']) != float:
            otr = line['Отрасль'].split(' / ')[:2]
            tmp += [otrasl_to_index[i] for i in otr]
            if len(otr) < 2:
                tmp += [0] * (2-len(otr))
        else:
            tmp += [otrasl_to_index['-1']] + [0]
        
        if type(line['Регион']) != float:
            tmp += [region_to_index[line['Регион']]]
        else:
            tmp += [region_to_index['-1']]
            
        if type(line['Организационно правовая форма']) != float:
            tmp += [forma_to_index[line['Организационно правовая форма']]]
        else:
            tmp += [forma_to_index['-1']]
            
        x.append(tmp)
        y.append(kbk_to_index[line['KBK']])
        
    if split is not None:
        return train_test_split(x, y, test_size=split, shuffle=True, stratify=y)
        
    return x, y

In [109]:
x_train, y_train = get_data(dfs)

In [111]:
model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=5)
model.fit(x_train, y_train)

KeyboardInterrupt: 